In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

# 1. Cargar y Ordenar (Siempre el paso 1)
df = pd.read_csv('../data/processed/f1_features_complete.csv')
df = df.sort_values(['year', 'round']).reset_index(drop=True)

feature_cols = [
    'pct_puntos_actual', 'pct_linear_points', 'posicion_media',
    'tendencia_ultimas_3', 'diff_con_lider_normalizada', 'progreso_temporada',
    'driver_quality_3y', 'team_avg_pos_3y', 'team_trend'
]
target_col = 'pct_puntos_final'

X = df[feature_cols]
y = df[target_col]

# 2. Configurar la Validación Cruzada Temporal
# Usamos lo mismo que validamos en el notebook 4.2
tscv = TimeSeriesSplit(n_splits=5)

# 3. Definir la rejilla de parámetros (Grid)
# Aquí ponemos los valores que queremos probar
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.1, 0.5, 1.0] # Regularización L1 (importante para evitar overfitting)
}

# 4. Configurar el Modelo Base
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42
)

# 5. Configurar la Búsqueda (GridSearchCV o RandomizedSearchCV)
print("Iniciando búsqueda de hiperparámetros...")

# Usamos MAE negativo porque Scikit-learn siempre intenta "maximizar" el score
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=tscv,           
    scoring=scorer, 
    verbose=1,
    n_jobs=-1
)

# 6. Ejecutar
grid_search.fit(X, y)

# 7. Resultados
print("\n" + "="*50)
print("MEJORES PARÁMETROS ENCONTRADOS")
print("="*50)
print(grid_search.best_params_)
print(f"\nMejor MAE Promedio: {-grid_search.best_score_:.4f}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 1728 candidates, totalling 8640 fits

MEJORES PARÁMETROS ENCONTRADOS
{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'subsample': 0.9}

Mejor MAE Promedio: 0.0106


In [2]:
df.columns

Index(['pct_puntos_actual', 'pct_linear_points', 'posicion_media',
       'tendencia_ultimas_3', 'diff_con_lider_normalizada',
       'progreso_temporada', 'year', 'round', 'driver', 'team',
       'team_normalized', 'driver_quality_3y', 'team_avg_pos_3y', 'team_trend',
       'pct_puntos_final'],
      dtype='object')